In [1]:
import gmplot, folium
from math import cos, pi, sqrt, tan, sin, atan, trunc
import numpy as np
import csv
#import matplotlib
#import matplotlib.pyplot as plt
import pandas as pd
#from scipy.misc import imread
#from matplotlib.backends.backend_pdf import PdfPages
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definitionen
file="C:\\he\python\\P1510-03\\1510-03c_Messdaten_190417.csv"

In [3]:
data_df = pd.read_csv(file,sep = ';')

In [4]:
data_df.head(5)

,RW,HW,ODL [nSv/h]
0,2567786,5717545,550
1,2567781,5717541,1000
2,2567839,5717543,300
3,2567852,5717543,250
4,2567977,5717531,260


In [5]:
"""
Converting a Gauss-Kruger-Code to a latitude / longitude coordinate.
"""
from math import cos, pi, sqrt, tan, sin, atan, trunc


def convert_GK_to_lat_long(right, height, use_wgs84=None):
    (x, y) = gauss_krueger_transformation(right, height)

    return seven_parameter_helmert_transf(x, y, use_wgs84)


def gauss_krueger_transformation(right, height):
    # Check for invalid Parameters
    if not ((right > 1000000) and (height > 1000000)):
        raise ValueError("No valid Gauss-Kruger-Code.")

    # Variables to prepare the geovalues
    GKRight = right
    GKHeight = height
    e2 = 0.0067192188
    c = 6398786.849
    rho = 180 / pi
    bII = (GKHeight / 10000855.7646) * (GKHeight / 10000855.7646)
    bf = 325632.08677 * (GKHeight / 10000855.7646) * ((((((0.00000562025 * bII + 0.00022976983) * bII - 0.00113566119) * bII + 0.00424914906) * bII - 0.00831729565) * bII + 1))

    bf /= 3600 * rho
    co = cos(bf)
    g2 = e2 * (co * co)
    g1 = c / sqrt(1 + g2)
    t = tan(bf)
    fa = (GKRight - trunc(GKRight / 1000000) * 1000000 - 500000) / g1

    GeoDezRight = ((bf - fa * fa * t * (1 + g2) / 2 + fa * fa * fa * fa * t * (5 + 3 * t * t + 6 * g2 - 6 * g2 * t * t) / 24) * rho)
    dl = fa - fa * fa * fa * (1 + 2 * t * t + g2) / 6 + fa * fa * fa * fa * fa * (1 + 28 * t * t + 24 * t * t * t * t) / 120
    GeoDezHeight = dl * rho / co + trunc(GKRight / 1000000) * 3

    return (GeoDezRight, GeoDezHeight)


def seven_parameter_helmert_transf(right, height, use_wgs84=False):
    earthRadius = 6378137  # Earth is a sphere with this radius
    aBessel = 6377397.155
    eeBessel = 0.0066743722296294277832
    ScaleFactor = 0.00000982
    RotXRad = -7.16069806998785E-06
    RotYRad = 3.56822869296619E-07
    RotZRad = 7.06858347057704E-06
    ShiftXMeters = 591.28
    ShiftYMeters = 81.35
    ShiftZMeters = 396.39
    LatitudeIt = 99999999

    if use_wgs84:
        ee = 0.0066943799
    else:
        ee = 0.00669438002290

    GeoDezRight = (right / 180) * pi
    GeoDezHeight = (height / 180) * pi

    n = eeBessel * sin(GeoDezRight) * sin(GeoDezRight)
    n = 1 - n
    n = sqrt(n)
    n = aBessel / n

    CartesianXMeters = n * cos(GeoDezRight) * cos(GeoDezHeight)
    CartesianYMeters = n * cos(GeoDezRight) * sin(GeoDezHeight)
    CartesianZMeters = n * (1 - eeBessel) * sin(GeoDezRight)

    CartOutputXMeters = (1 + ScaleFactor) * CartesianXMeters + RotZRad * CartesianYMeters - RotYRad * CartesianZMeters + ShiftXMeters
    CartOutputYMeters = -1 * RotZRad * CartesianXMeters + (1 + ScaleFactor) * CartesianYMeters + RotXRad * CartesianZMeters + ShiftYMeters
    CartOutputZMeters = RotYRad * CartesianXMeters - RotXRad * CartesianYMeters + (1 + ScaleFactor) * CartesianZMeters + ShiftZMeters

    GeoDezHeight = atan(CartOutputYMeters / CartOutputXMeters)

    Latitude = (CartOutputXMeters * CartOutputXMeters) + (CartOutputYMeters * CartOutputYMeters)
    Latitude = sqrt(Latitude)
    Latitude = CartOutputZMeters / Latitude
    Latitude = atan(Latitude)

    not_accurate_enough = True

    while not_accurate_enough:
        LatitudeIt = Latitude

        n = 1 - ee * sin(Latitude) * sin(Latitude)
        n = sqrt(n)
        n = earthRadius / n

        Latitude = CartOutputXMeters * CartOutputXMeters + CartOutputYMeters * CartOutputYMeters
        Latitude = sqrt(Latitude)
        Latitude = (CartOutputZMeters + ee * n * sin(LatitudeIt)) / Latitude
        Latitude = atan(Latitude)

        not_accurate_enough = abs(Latitude - LatitudeIt) >= 0.000000000000001

    GeoDezRight = (Latitude / pi) * 180
    GeoDezHeight = (GeoDezHeight) / pi * 180

    return (GeoDezRight, GeoDezHeight)

In [6]:
#!dir

In [7]:
def wrapper_conv_Lat(dframe):    
      return convert_GK_to_lat_long(dframe['RW'],dframe['HW'],use_wgs84=True)[0]
    
def wrapper_conv_Lon(dframe):    
      return convert_GK_to_lat_long(dframe['RW'],dframe['HW'],use_wgs84=True)[1]

In [8]:
data_df['Latitude'] = data_df.apply(wrapper_conv_Lat, axis=1);
data_df['Longitude'] = data_df.apply(wrapper_conv_Lon, axis=1);
data_df['Messpunkt ODL'] = data_df['ODL [nSv/h]'].astype(str)

In [9]:
# Place map
gmap = gmplot.GoogleMapPlotter(data_df['Latitude'].mean(),data_df['Longitude'].mean(), 18)

# Scatter points
gmap.scatter(data_df['Latitude'], data_df['Longitude'], '#3B0B39', size=2, marker=False)

#tut nicht
gmap.marker(data_df['Latitude'][0], data_df['Longitude'][0], 'cornflowerblue')

# Draw
gmap.draw("C:\\Users\\he\\Desktop\\Zweckel.html")

In [10]:
data_df.to_csv("C:\\Users\\he\\Desktop\\Zweckel_df-data_190424.csv",sep = ';')

In [11]:
locations = data_df[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist

[[51.58825370565419, 6.977439511317778],
 [51.58821835948112, 6.977366606654291],
 [51.58822936165242, 6.978203736850894],
 [51.588227798304985, 6.978391282976138],
 [51.58810491018252, 6.980192289116456],
 [51.58805732537165, 6.98050870635819],
 [51.58781631515051, 6.981383691231336],
 [51.58747680679235, 6.982213255536174],
 [51.587084822267386, 6.9828685286288605],
 [51.587030539518764, 6.982910643625053],
 [51.58691226223392, 6.9830812364739865],
 [51.58674917199511, 6.983236432199969]]

In [12]:
map = folium.Map(location=[data_df['Latitude'].mean(),data_df['Longitude'].mean()], zoom_start=18)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=data_df['Messpunkt ODL'][point]).add_to(map)
map
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel.html")

In [13]:
map = folium.Map(location=[data_df['Latitude'].mean(),data_df['Longitude'].mean()], zoom_start=18)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=folium.Popup(folium.Html("""
            </b> {0} </br>
            </b> nSv/h
            """.format(data_df['Messpunkt ODL'][point]), script=True), max_width='100%', show=True, sticky=True)).add_to(map)
        
        #str(data_df['Messpunkt ODL'][point])+' nSv/h', parse_html=True, max_width='100%', show=True)).add_to(map)
map
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel2.html")

In [45]:
map = folium.Map(location=[data_df['Latitude'].mean(),data_df['Longitude'].mean()], zoom_start=18)
for point in range(0, len(locationlist)):
    folium.Circle(locationlist[point], 2, fill=True).add_child(folium.Popup(folium.Html("""
            </b> {0} </br>
            </b> nSv/h
            """.format(data_df['Messpunkt ODL'][point]), script=True), max_width='100%', show=True, sticky=True)).add_to(map)
map
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel3.html")
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel3.png")

In [15]:
map = folium.Map(location=[data_df['Latitude'].mean(),data_df['Longitude'].mean()], zoom_start=18)
for point in range(0, len(locationlist)):
    folium.CircleMarker(location=locationlist[point],radius=10, color='blue', 
                        popup='{0} nSv/h'.format(data_df['Messpunkt ODL'][point]),
    fill=True).add_to(map)
map
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel4.html")

In [46]:
from folium.features import DivIcon
map = folium.Map(location=[data_df['Latitude'].mean(),data_df['Longitude'].mean()], zoom_start=18)
for point in range(0, len(locationlist)):
    #folium.Circle(locationlist[point], radius=5, fill=True).add_to(map)
    folium.CircleMarker(locationlist[point], radius=12, fill=True).add_to(map)
    folium.map.Marker(locationlist[point], icon=DivIcon(
        icon_size=(30,12),
        icon_anchor=(10,10),#muss gleich der Font-Size sein für mittige Darstellung
        html='<div style="font-size: 10pt; color : red" </b>{0}</b> </div>'.format(data_df['Messpunkt ODL'][point]))).add_to(map)
map
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel5.html")
map.save(outfile= "C:\\Users\\he\\Desktop\\Messpunkte-GWL_Zweckel5.png")